## Writing xarray back to GRIB

In [1]:
import earthkit.data
import numpy as np

# load pressure level GRIB data on regular_ll grid
fname = "pl_regular_ll.grib"
earthkit.data.download_example_file(fname, remote_dir="test-data/xr_engine/level")
ds_ek = earthkit.data.from_source("file", fname)

In [2]:
ds = ds_ek.to_xarray()
ds

<xarray.Dataset> Size: 1MB
Dimensions:                  (forecast_reference_time: 4, step: 2, levelist: 6,
                              latitude: 19, longitude: 36)
Coordinates:
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 32B 202...
  * step                     (step) timedelta64[ns] 16B 00:00:00 06:00:00
  * levelist                 (levelist) int64 48B 300 400 500 700 850 1000
  * latitude                 (latitude) float64 152B 90.0 80.0 ... -80.0 -90.0
  * longitude                (longitude) float64 288B 0.0 10.0 ... 340.0 350.0
Data variables:
    r                        (forecast_reference_time, step, levelist, latitude, longitude) float64 263kB ...
    t                        (forecast_reference_time, step, levelist, latitude, longitude) float64 263kB ...
    u                        (forecast_reference_time, step, levelist, latitude, longitude) float64 263kB ...
    v                        (forecast_reference_time, step, levelist, latitude, longitude) float64 263kB ...
    z                        (forecast_reference_time, step, levelist, latitude, longitude) float64 263kB ...
Attributes:
    param:        z
    class:        od
    stream:       oper
    levtype:      pl
    type:         fc
    expver:       0001
    domain:       g
    number:       0
    Conventions:  CF-1.8
    institution:  ECMWF

Modify values. We must ensure keep_attrs=True for the GRIB metadata attached to the dataset to be copied over.

In [3]:
import xarray as xr 
xr.set_options(keep_attrs=True)

In [4]:
ds += 1

Convert one variable back into a GRIB fieldlist (will be stored in memory).

In [5]:
r = ds["t"].earthkit.to_fieldlist()

In [6]:
len(r)

48

In [7]:
r.head()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,300,20240603,0,0,fc,0,regular_ll
1,ecmf,t,isobaricInhPa,400,20240603,0,0,fc,0,regular_ll
2,ecmf,t,isobaricInhPa,500,20240603,0,0,fc,0,regular_ll
3,ecmf,t,isobaricInhPa,700,20240603,0,0,fc,0,regular_ll
4,ecmf,t,isobaricInhPa,850,20240603,0,0,fc,0,regular_ll


Convert back the whole dataset into a GRIB fieldlist (will be stored on memory).

In [8]:
r = ds.earthkit.to_fieldlist()
r.head()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,r,isobaricInhPa,300,20240603,0,0,fc,0,regular_ll
1,ecmf,r,isobaricInhPa,400,20240603,0,0,fc,0,regular_ll
2,ecmf,r,isobaricInhPa,500,20240603,0,0,fc,0,regular_ll
3,ecmf,r,isobaricInhPa,700,20240603,0,0,fc,0,regular_ll
4,ecmf,r,isobaricInhPa,850,20240603,0,0,fc,0,regular_ll


In [9]:
len(r)

240

Save grib data to disk.

In [10]:
out_name = "_from_xr.grib"
r.save(out_name)
ds1 = earthkit.data.from_source("file", out_name)
len(ds1)

240

Save the xarray object straight into GRIB. This method does not require the creation of a fieldlist (more memory efficient).

In [11]:
out_name = "_from_xr_1.grib"
ds.earthkit.to_grib(out_name)
ds1 = earthkit.data.from_source("file", out_name)
len(ds1)

240

Check if the computation results were correctly written to the output GRIB data.

In [12]:
# original GRIB data
ds_ek.sel(param="t", step=0, level=500)[0].values[0]

250.22500610351562

In [13]:
# GRIB data save from the modified xarray object
ds1.sel(param="t", step=0, level=500)[0].values[0]

251.22500610351562